# Introduction to Mesa

* [GitHub](https://github.com/projectmesa)
* [API](https://mesa.readthedocs.io/en/latest/apis/api_main.html)
* [Overview tutorial](https://mesa.readthedocs.io/latest/overview.html)

## Mesa Modules

### Agent

In [ ]:
import mesa

class MyAgent(mesa.Agent):
    def __init__(self, model, age):
        super().__init__(model)
        self.age = age

    def step(self):
        self.age += 1
        print(f"Agent {self.unique_id} now is {self.age} years old")
        # Whatever else the agent does when activated

In [ ]:
import mesa

class MyAgent(mesa.Agent):
    def __init__(self, model, age):
        super().__init__(model)
        self.age = age

    def step(self):
        self.age += 1
        if self.model.debug:
            print(f"Agent {self.unique_id} now is {self.age} years old")
        # Whatever else the agent does when activated

    def __str__(self):
        return f"Agent {self.unique_id}"
        
    def __repr__(self):
        return f"Agent {self.unique_id} ({self.age} years old)"

### Model

In [ ]:
class MyModel(mesa.Model):
    debug = False
    def __init__(self, n_agents):
        super().__init__()
        self.grid = mesa.space.MultiGrid(10, 10, torus=True)
        for _ in range(n_agents):
            initial_age = self.random.randint(0, 80)
            a = MyAgent(self, initial_age)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)

    def step(self):
        self.agents.shuffle_do("step")

In [ ]:
model = MyModel(n_agents=7)

### Space: Discrete and continiuous
-> Covered in later lecture

## AgentSet and model.agents

`model.agents` is an AgentSet containing all agents in the model. It’s automatically updated when agents are added or removed:

In [ ]:
# Get total number of agents
num_agents = len(model.agents)

# Iterate over all agents
for agent in model.agents:
    print(agent)

#### Selecting: Filter agents based on criteria.

In [ ]:
high_energy_agents = model.agents.select(lambda a: a.age > 70)
print(high_energy_agents)

In [ ]:
high_energy_agents = model.agents.select(lambda a: a.age > 70)
selected_list = [agent for agent in high_energy_agents]
print(selected_list)

#### Shuffling and Sorting: Randomize or order agents.

In [ ]:
shuffled_agents = model.agents.shuffle()
sorted_agents = model.agents.sort(key="energy", ascending=False)

#### Applying methods: Execute methods on all agents.

In [ ]:
model.agents.do("step")
model.agents.shuffle_do("move")  # Shuffle then apply method

#### Aggregating: Compute aggregate values across agents.

In [ ]:
avg_energy = model.agents.agg("energy", func=np.mean)

#### Grouping: Group agents by attributes.

In [ ]:
grouped_agents = model.agents.groupby("species")

for _, agent_group in grouped_agents:
   agent_group.shuffle_do()
species_counts = grouped_agents.count()
mean_age_by_group = grouped_agents.agg("age", np.mean)

## Time Advancement and Agent Activation

### Basic Time Steps

Run the model for a specified number of steps:

In [ ]:
model = MyModel(n_agents=7)
for _ in range(100):
    model.step()

### Agent Activation Patterns

In [ ]:
# Sequential activation
model.agents.do("step")

In [ ]:
# Random activation
model.agents.shuffle_do("step")

## Analysis modules

Data collection and batch running are implemented in the appropriately-named analysis modules:

* [mesa.datacollection](https://mesa.readthedocs.io/latest/apis/datacollection.html)
* [mesa.batchrunner](https://mesa.readthedocs.io/latest/apis/batchrunner.html)

In [ ]:
import mesa
import numpy as np

# ...

class MyModel(mesa.Model):
    debug = False
    def __init__(self):
        n_agents = 7
        super().__init__()
        self.grid = mesa.space.MultiGrid(10, 10, torus=True)
        for _ in range(n_agents):
            initial_age = self.random.randint(0, 80)
            a = MyAgent(self, initial_age)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)
        self.datacollector = mesa.DataCollector(
            model_reporters={"mean_age": lambda m: m.agents.agg("age", np.mean)},
            agent_reporters={"age": "age"}
        )

    def step(self):
        self.agents.shuffle_do("step")
        self.datacollector.collect(self)

After you’re done running it, you can extract the data as a pandas DataFrame:

In [ ]:
model = MyModel(5)
for t in range(10):
    model.step()
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

To batch-run the model while varying, for example, the n_agents parameter, you’d use the `batch_run` function:

In [ ]:
import mesa

parameters = {"n_agents": range(1, 6)}
results = mesa.batch_run(
    MyModel,
    parameters,
    iterations=5,
    max_steps=100,
    data_collection_period=1,
    number_processes=1  # Change to use multiple CPU cores for parallel execution
)

## Visualization
SolaraViz allows for interactive, customizable visualizations of your models.

In [ ]:
from mesa.visualization import SolaraViz, make_space_component, make_plot_component


def agent_portrayal(agent):
    return {"color": "blue", "size": 50}


model_params = {
    "n_agents": Slider(
        label="Number of agents:",
        value=50,
        min=1,
        max=100,
        step=1
    )
}

model = MyModel()

page = SolaraViz(
    model,
    components = [
        make_space_component(agent_portrayal),
        make_plot_component("mean_age")
    ],
    model_params=model_params,
)
page

## Example: Schelling Model

In [ ]:
import solara

from mesa.examples.basic.schelling.model import Schelling
from mesa.visualization import (
    Slider,
    SolaraViz,
    make_plot_component,
    make_space_component,
)


def get_happy_agents(model):
    """Display a text count of how many happy agents there are."""
    return solara.Markdown(f"**Happy agents: {model.happy}**")


def agent_portrayal(agent):
    return {"color": "tab:orange" if agent.type == 0 else "tab:blue"}


model_params = {
    "seed": {
        "type": "InputText",
        "value": 42,
        "label": "Random Seed",
    },
    "density": Slider("Agent density", 0.8, 0.1, 1.0, 0.1),
    "minority_pc": Slider("Fraction minority", 0.2, 0.0, 1.0, 0.05),
    "homophily": Slider("Homophily", 0.4, 0.0, 1.0, 0.125),
    "width": 20,
    "height": 20,
}

model1 = Schelling()

HappyPlot = make_plot_component({"happy": "tab:green"})

page = SolaraViz(
    model1,
    components=[
        make_space_component(agent_portrayal),
        HappyPlot,
        get_happy_agents,
    ],
    model_params=model_params,
)
page  # noqa


## Introduction to Evacuation Model

In [ ]:
import sys
sys.path.insert(0,'../../abmodel')

from mesa.visualization import SolaraViz, make_space_component, make_plot_component
from fire_evacuation.model import FireEvacuation
from fire_evacuation.agent import Human, FireExit, Wall, Sight
import os
import solara

current_dir = '../../abmodel'

# Specify the parameters changeable by the user, in the web interface
model_params = {
    "random_spawn": {
        "type": "Checkbox",
        "value": True,
        "label": "Random spawn of initial positions",
    },
    "floor_size": {
        "type": "SliderInt",
        "value": 12,
        "label": "Room size (edge)",
        "min": 5,
        "max": 30,
        "step": 1,
    },
    "human_count": {
        "type": "SliderInt",
        "value": 80,
        "label": "Number Of Human Agents",
        "min": 1,
        "max": 500,
        "step": 5,
    },
    "max_speed": {
        "type": "SliderInt",
        "value": 2,
        "label": "Maximum Speed of agents",
        "min": 1,
        "max": 5,
        "step": 1,
    },
    "alarm_believers_prop": {
        "type": "SliderFloat",
        "value": 1.0,
        "label": "Proportion of Alarm Believers",
        "min": 0.0,
        "max": 1.0,
        "step": 0.05,
    },
    "cooperation_mean": {
        "type": "SliderFloat",
        "value": 0.3,
        "label": "Mean Cooperation",
        "min": 0.0,
        "max": 1.0,
        "step": 0.01,
    },
    "nervousness_mean": {
        "type": "SliderFloat",
        "value": 0.3,
        "label": "Mean Nervousness",
        "min": 0.0,
        "max": 1.0,
        "step": 0.01,
    },
}

def agent_portrayal(agent):
    size = 10
    if type(agent) is Human:
        if agent.believes_alarm:
            # believes in alarm
            shape = os.path.join(current_dir, "fire_evacuation/resources/alarmbeliever.png")
        elif agent.nervousness > Human.NERVOUSNESS_PANIC_THRESHOLD:
            shape = os.path.join(current_dir, "fire_evacuation/resources/panicked_human.png")
        elif agent.humantohelp is not None:
            shape = os.path.join(current_dir, "fire_evacuation/resources/cooperating_human.png")
        else:
            shape = os.path.join(current_dir, "fire_evacuation/resources/human.png")
    elif type(agent) is FireExit:
        shape = os.path.join(current_dir, "fire_evacuation/resources/fire_exit.png")
    elif type(agent) is Wall:
        shape = os.path.join(current_dir, "fire_evacuation/resources/wall.png")
    elif type(agent) is Sight:
        shape = os.path.join(current_dir, "fire_evacuation/resources/eye.png")
    else:
        shape = "X"
    return {"size": size,
            "marker": shape,
            "color": "red",
            }

model = solara.reactive(FireEvacuation(
            floor_size = 14,
            human_count = 70,
            alarm_believers_prop = 1.0,
            max_speed = 2,
            seed = 3)
        )

page = SolaraViz(
    model,
    model_params = model_params,
    name="Evacuation Model",
    components=[make_space_component(agent_portrayal),
                make_plot_component("AvgNervousness"),
                ],
)

page  # noqa